<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-classification-data" data-toc-modified-id="Load-classification-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load classification data</a></span></li><li><span><a href="#Remove-FOVs-which-are-corrupted-(round-2!)" data-toc-modified-id="Remove-FOVs-which-are-corrupted-(round-2!)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Remove FOVs which are corrupted (round 2!)</a></span></li><li><span><a href="#t-SNE-plot-(requires-normalization)" data-toc-modified-id="t-SNE-plot-(requires-normalization)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>t-SNE plot (requires normalization)</a></span></li><li><span><a href="#Plot-R/B-vs-G/B-(without-normalization)" data-toc-modified-id="Plot-R/B-vs-G/B-(without-normalization)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot R/B vs G/B (without normalization)</a></span></li><li><span><a href="#Plot-histogram-of-total-intensity-for-all-whole-blood-slides" data-toc-modified-id="Plot-histogram-of-total-intensity-for-all-whole-blood-slides-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plot histogram of total intensity for all whole blood slides</a></span></li><li><span><a href="#Plot-histogram-of-size--for-all-whole-blood-slides" data-toc-modified-id="Plot-histogram-of-size--for-all-whole-blood-slides-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Plot histogram of size  for all whole blood slides</a></span></li><li><span><a href="#Create-cross-validation-folds" data-toc-modified-id="Create-cross-validation-folds-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Create cross-validation folds</a></span></li><li><span><a href="#Train-model-using-cross-validation" data-toc-modified-id="Train-model-using-cross-validation-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Train model using cross-validation</a></span></li><li><span><a href="#Plot-results" data-toc-modified-id="Plot-results-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Plot results</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Load-results" data-toc-modified-id="Load-results-9.0.1"><span class="toc-item-num">9.0.1&nbsp;&nbsp;</span>Load results</a></span></li><li><span><a href="#Show-results" data-toc-modified-id="Show-results-9.0.2"><span class="toc-item-num">9.0.2&nbsp;&nbsp;</span>Show results</a></span></li><li><span><a href="#Identify-which-AUC-curve-is-the-closest-to-the-mean-one-(e.g.-compare-AUCs-to-mean-AUC)" data-toc-modified-id="Identify-which-AUC-curve-is-the-closest-to-the-mean-one-(e.g.-compare-AUCs-to-mean-AUC)-9.0.3"><span class="toc-item-num">9.0.3&nbsp;&nbsp;</span>Identify which AUC curve is the closest to the mean one (e.g. compare AUCs to mean AUC)</a></span><ul class="toc-item"><li><span><a href="#FPR-vs-FNR-curves" data-toc-modified-id="FPR-vs-FNR-curves-9.0.3.1"><span class="toc-item-num">9.0.3.1&nbsp;&nbsp;</span>FPR vs FNR curves</a></span></li></ul></li></ul></li></ul></li></ul></div>

In [1]:
import os
import numpy as np
import pandas as pd
import csv   
import cv2 as cv

from sklearn import preprocessing
from scipy.stats import gaussian_kde
from sklearn import metrics

import matplotlib.pylab as plt
%matplotlib inline
import matplotlib.pyplot as mpplt
import matplotlib.image as mpimg
cmap = plt.cm.rainbow
import matplotlib
norm = matplotlib.colors.Normalize(vmin=0., vmax=1.)


# Load classification data

In [8]:
# Crop image
x_low = 821
x_high = 1643
y_low = 819
y_high = 1640
crop_box = [x_low,x_high,y_low,y_high]

# Top-hat filtering kernel
disk_diameter = 17
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (int(disk_diameter), int(disk_diameter)))

In [9]:
path_to_load_data = os.path.join('..','data','output_step_3_combine_fluo_and_brightfield_features')

X_with_helpers_with_overlap = pd.read_csv(os.path.join(path_to_load_data,'df_with_helper_with_overlap.csv'),index_col=0)
y = np.genfromtxt( os.path.join(path_to_load_data,'y.csv') , delimiter=',')

print('df_with_helper_with_overlap',X_with_helpers_with_overlap.shape)
print('y',y.shape)
print('y',np.unique(y,return_counts=True))

('df_with_helper_with_overlap', (873, 54))
('y', (873,))
('y', (array([0., 1.]), array([549, 324])))


# Remove FOVs which are corrupted (round 2!)

In [10]:
# FOVs to remove
FOV_to_remmove = {
    '1128-w-16': ['_0002_0001_fluorescent.jpeg','_0006_0004_fluorescent.jpeg','_0009_0004_fluorescent.jpeg','_0010_0002_fluorescent.jpeg'],
    '1128-w-17': ['_0006_0000_fluorescent.jpeg','_0006_0001_fluorescent.jpeg','_0007_0002_fluorescent.jpeg','_0008_0010_fluorescent.jpeg','_0009_0011_fluorescent.jpeg','_0010_0004_fluorescent.jpeg','_0010_0005_fluorescent.jpeg','_0010_0007_fluorescent.jpeg','_0012_0008_fluorescent.jpeg','_0012_0010_fluorescent.jpeg','_0012_0011_fluorescent.jpeg','_0013_0009_fluorescent.jpeg'],
    '1128-w-18': ['_0001_0010_fluorescent.jpeg','_0003_0011_fluorescent.jpeg','_0005_0004_fluorescent.jpeg','_0008_0011_fluorescent.jpeg','_0009_0000_fluorescent.jpeg','_0010_0009_fluorescent.jpeg','_0011_0009_fluorescent.jpeg','_0011_0010_fluorescent.jpeg','_0011_0011_fluorescent.jpeg'],
    '1128-w-19': ['_0001_0008_fluorescent.jpeg','_0003_0005_fluorescent.jpeg','_0004_0008_fluorescent.jpeg','_0005_0008_fluorescent.jpeg','_0007_0000_fluorescent.jpeg','_0007_0006_fluorescent.jpeg','_0007_0009_fluorescent.jpeg','_0009_0003_fluorescent.jpeg','_0009_0004_fluorescent.jpeg'],
    '1128-w-20': ['_0001_0002_fluorescent.jpeg','_0001_0008_fluorescent.jpeg','_0003_0008_fluorescent.jpeg','_0005_0009_fluorescent.jpeg','_0007_0006_fluorescent.jpeg','_0007_0009_fluorescent.jpeg','_0007_0011_fluorescent.jpeg','_0008_0004_fluorescent.jpeg','_0009_0010_fluorescent.jpeg'],
    '1128-w-31': ['_0001_0009_fluorescent.jpeg','_0004_0007_fluorescent.jpeg'],
    '1128-w-32': ['_0003_0001_fluorescent.jpeg', '_0008_0004_fluorescent.jpeg','_0008_0006_fluorescent.jpeg'],
    '1128-w-41': ['_0001_0007_fluorescent.jpeg', '_0001_0009_fluorescent.jpeg', '_0001_0011_fluorescent.jpeg','_0002_0008_fluorescent.jpeg','_0006_0004_fluorescent.jpeg','_0011_0005_fluorescent.jpeg','_0011_0009_fluorescent.jpeg','_0012_0009_fluorescent.jpeg'],
    '1128-w-42': ['_0003_0006_fluorescent.jpeg','_0006_0002_fluorescent.jpeg','_0007_0000_fluorescent.jpeg','_0008_0003_fluorescent.jpeg'],
    '1128-w-43': ['_0003_0001_fluorescent.jpeg','_0004_0005_fluorescent.jpeg','_0005_0006_fluorescent.jpeg','_0006_0001_fluorescent.jpeg','_0006_0008_fluorescent.jpeg','_0007_0001_fluorescent.jpeg','_0007_0003_fluorescent.jpeg','_0007_0011_fluorescent.jpeg','_0009_0001_fluorescent.jpeg','_0010_0007_fluorescent.jpeg','_0012_0006_fluorescent.jpeg','_0012_0011_fluorescent.jpeg','_0013_0010_fluorescent.jpeg','_0014_0010_fluorescent.jpeg'],
}

In [11]:
idx_to_remove_global = [False]*X_with_helpers_with_overlap.shape[0]

# Iterate over patients
for patient_id, FOV_ids in FOV_to_remmove.iteritems():
    # Iterate over FOVs
    for FOV_id in FOV_ids: 
        # Identify spots corresponding to patients+FOVs to remove
        idx_to_remove_1 = (X_with_helpers_with_overlap['foldername'].values==patient_id)
        idx_to_remove_2 = (X_with_helpers_with_overlap['filename'].values==FOV_id)
        idx_to_remove = np.logical_and(idx_to_remove_1,idx_to_remove_2)
        idx_to_remove_global = np.logical_or(idx_to_remove_global,idx_to_remove)
        
# Remove spots
X_with_helpers_with_overlap = X_with_helpers_with_overlap[~idx_to_remove_global].reset_index(drop=True)
y = y[~idx_to_remove_global]

#print('X_with_overlap',X_with_overlap.shape)
print('X_with_helpers_with_overlap',X_with_helpers_with_overlap.shape)
print('y',y.shape)
print('y',np.unique(y,return_counts=True))

('X_with_helpers_with_overlap', (873, 54))
('y', (873,))
('y', (array([0., 1.]), array([549, 324])))


In [12]:
cols_to_keep = [x for x in list(X_with_helpers_with_overlap.columns) if x not in ['filename','foldername','spot_idx'] ]

X_with_overlap = X_with_helpers_with_overlap[cols_to_keep]
print('X_with_helpers_with_overlap',X_with_helpers_with_overlap.shape)
print('X_with_overlap',X_with_overlap.shape)


('X_with_helpers_with_overlap', (873, 54))
('X_with_overlap', (873, 51))


# t-SNE plot (requires normalization)

In [13]:
from sklearn.manifold import TSNE

In [14]:
X_for_tsne = np.copy(X_with_overlap)
y_for_tsne = np.copy(y)
# Shuffle data
idx_to_shuffle = np.random.permutation(len(y_for_tsne))
X_for_tsne = X_for_tsne[idx_to_shuffle]
y_for_tsne = y_for_tsne[idx_to_shuffle]
print('X_for_tsne',X_for_tsne.shape)
print('y_for_tsne',y_for_tsne.shape)
# Scale + center data for t-SNE
X_for_tsne_scaled = preprocessing.scale(X_for_tsne)
print('X_for_tsne_scaled',X_for_tsne_scaled.shape)



('X_for_tsne', (873, 51))
('y_for_tsne', (873,))
('X_for_tsne_scaled', (873, 51))


In [15]:
# Extract 10k fluo spots from each class, for t-SNE plots
n_spots_for_tsne = 10000
np.random.seed(0)

idx_ = np.random.permutation(n_spots_for_tsne)

idx_tsne_0 = np.where(y_for_tsne==0.)[0][idx_]
idx_tsne_1 = np.where(y_for_tsne==1.)[0][idx_]
print('idx_tsne_0',idx_tsne_0)
print('idx_tsne_1',idx_tsne_1)

IndexError: index 9394 is out of bounds for axis 0 with size 549

In [ ]:
idx_tsne = np.array(list(idx_tsne_0) + list(idx_tsne_1))
print('idx_tsne',idx_tsne)

In [ ]:
X_for_tsne_scaled = X_for_tsne_scaled[idx_tsne]
y_for_tsne = y_for_tsne[idx_tsne]
print('X_for_tsne_scaled',X_for_tsne_scaled.shape)
print('y_for_tsne',y_for_tsne.shape)


In [ ]:
# Run t-SNE 
X_for_tsne_ = TSNE(n_components=2, random_state=6).fit_transform(X_for_tsne_scaled)

In [ ]:
x_PF = X_for_tsne_[:,0][y_for_tsne==1.]
x_WB = X_for_tsne_[:,0][y_for_tsne==0.]
print('x_PF',x_PF.shape)
print('x_WB',x_WB.shape)

y_PF = X_for_tsne_[:,1][y_for_tsne==1.]
y_WB = X_for_tsne_[:,1][y_for_tsne==0.]
print('y_PF',y_PF.shape)
print('y_WB',y_WB.shape)

# dscatter 
# http://localhost:8000/notebooks/Box/Malariascope_Paper_TeamFolder/Maxime/Malaria%20Detection/classification/main_classification.ipynb#
xy_PF = np.vstack([x_PF,y_PF])
z_PF = gaussian_kde(xy_PF)(xy_PF)
xy_WB = np.vstack([x_WB,y_WB])
z_WB = gaussian_kde(xy_WB)(xy_WB)
plt.figure(figsize=(10,10))
plt.scatter(x_PF, y_PF, c=z_PF, s=8, edgecolor='',marker='o',cmap='autumn')
plt.scatter(x_WB, y_WB, c=z_WB, s=8, edgecolor='',marker='o',cmap='winter')
#plt.show()
plt.savefig('tsne_v2.png')


In [ ]:
"""# Save t-sne data to disk
np.savetxt("X_for_tsne_v2_.csv", X_for_tsne_, delimiter=",")
np.savetxt("y_for_tsne_v2_.csv", y_for_tsne, delimiter=",")
"""

# Plot R/B vs G/B (without normalization) 

In [ ]:
print('X_with_overlap',X_with_overlap.columns)

In [ ]:
# Get R/B: is it a feature already? or should i compute it?

In [ ]:
R_ = X_with_overlap['R_int'].values
B_ = X_with_overlap['B_int'].values

R_over_B = R_/(B_+0.1) #+0.1 can be deleted if B_ doesnt have a 0...
plt.hist(R_over_B, normed=True, bins=70)
plt.ylabel('Probability')
plt.xlim(0.,2.3)
plt.show()

In [ ]:
# Get G/B: is it a feature already? or should i compute it?

In [ ]:
G_over_B = X_with_overlap['G_int/B_int'].values
print('G_over_B',G_over_B)

plt.hist(G_over_B, normed=True, bins=70)
plt.ylabel('Probability')

plt.xlim(0.,2.3)


In [ ]:
# Plot R/B vs G/B. Color code for y=0 and y=1
#print('G_over_B',G_over_B.shape)
#print('R_over_B',R_over_B.shape)
idx_0 = (y==0)
G_over_B_0 = G_over_B[idx_0]
R_over_B_0 = R_over_B[idx_0]
print('G_over_B_0',G_over_B_0.shape)
print('R_over_B_0',R_over_B_0.shape)
idx_1 = (y==1)
G_over_B_1 = G_over_B[idx_1]
R_over_B_1 = R_over_B[idx_1]
print('G_over_B_1',G_over_B_1.shape)
print('R_over_B_1',R_over_B_1.shape)

plt.scatter(G_over_B_0, R_over_B_0, s=0.005, c='green',alpha=0.7) #, s=area, c=colors, alpha=0.5)
plt.scatter(G_over_B_1, R_over_B_1, s=0.005, c='red',alpha=0.7) #, s=area, c=colors, alpha=0.5)
plt.title('Scatter plot R_over_B vs G_over_B')
plt.xlabel('G_over_B')
plt.ylabel('R_over_B')
plt.show()




# Plot histogram of total intensity for all whole blood slides

In [ ]:
X_with_overlap.columns

In [ ]:
# Get sumRBG_int
sumRBG_int = X_with_overlap['sumRBG_int'].values
# Get WB slides 
idx_WB = (y==0)
print('idx_WB',np.sum(idx_WB))
# Keep sumRBG_int for WB slides
sumRBG_int_WB = sumRBG_int[idx_WB]

plt.hist(sumRBG_int_WB, normed=True, bins=70)
plt.ylabel('Probability')
plt.title('Histogram of total spot intensity for all whole blood slides')

#plt.xlim(0.,2.3)


# Plot histogram of size  for all whole blood slides

In [ ]:
"""# Get sumRBG_int
sumRBG_int = X_with_overlap['sumRBG_int'].values
# Get sumRBG_int/NumPix
sumRBG_int_over_NumPix = X_with_overlap['sumRBG_int/NumPix'].values
# Get NumPix
NumPix = sumRBG_int/sumRBG_int_over_NumPix

# Get WB slides 
idx_WB = (y==0)
print('idx_WB',np.sum(idx_WB))
# Keep NumPix for WB slides
NumPix_WB = NumPix[idx_WB]

plt.hist(NumPix_WB, normed=True, bins=100)
plt.ylabel('Probability')
plt.title('Histogram of spot size for all whole blood slides')
plt.yscale('log')
#plt.xlim(0.,2.3)
"""

# Create cross-validation folds

In [ ]:
np.random.seed(0)
n_folds = 20

# List patients
slides_PF = ['1128-p-63', '1128-p-64', '1128-p-66', '1128-p-67', '1128-p-74', '1128-p-75', '1128-p-76b', '1128-p-79']
slides_WB = ['1128-w-16', '1128-w-17', '1128-w-18', '1128-w-19', '1128-w-20', '1128-w-31', '1128-w-32', '1128-w-41', '1128-w-42', '1128-w-43']
print('slides_PF',len(slides_PF))
print('slides_WB',len(slides_WB))

folds = {}
for i in range(n_folds): 
    # Pick 3 PF slides
    fold_slides_PF = list(np.random.choice(slides_PF, size=3, replace=False))
    # Pick 3 WB slides
    fold_slides_WB = list(np.random.choice(slides_WB, size=3, replace=False))
    folds['fold_'+str(i)] = fold_slides_PF + fold_slides_WB
    
# Get idx corresponding to each fold
fold_idx = {}
# Iterate over folds
for fold, fold_folders in folds.iteritems():
    # Get test idx 
    test_idx = X_with_helpers_with_overlap['foldername'].isin(fold_folders)
    fold_idx[fold] = test_idx


# Train model using cross-validation

We iterate over folds. For each fold: 
- We get train data and test data
- We normalize train and test data seperately
- Shuffle the rows of train data and test data, respectively
- We train on train set, and evaluate and test set

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
np.random.seed(0)

In [ ]:
params = { 
    'learning_rate':0.10, #0.10 
    'n_estimators': 233, #233, 
    'max_depth':6, # 6 
    'min_child_weight': 10, #10 
    'gamma': 1., # 1. 
    'subsample':1., # 1. 
    'colsample_bytree':1., #1. 
    'objective':'binary:logistic',
    'nthread':-1,
    'scale_pos_weight':1., #1. 
    'seed':2, # 2
    'reg_lambda':30., # 30. 
    'reg_alpha':0., # 0. 
}


In [ ]:
columns_to_keep = [x for x in list(X_with_helpers_with_overlap.columns.values) if x not in ['filename','foldername', 'spot_idx'] ]
X_ = X_with_helpers_with_overlap[columns_to_keep]
print('X_with_helpers_with_overlap',X_with_helpers_with_overlap.shape)
print('X_',X_.shape)
print('y',y.shape)

In [ ]:
fnrs_global,fprs_global,thresholds_global,fold_global,auc_global,fpr_value_global,fnr_value_global = [],[],[],[],[],[],[]

# Iterate over folds
for fold, test_idx in fold_idx.iteritems():
    print('======== ',fold)
    foldername_k_global,filename_k_global,spot_idx_k_global,overlap_k_global = [],[],[],[]
    
    # Get train data
    X_train = X_[~test_idx]
    X_train = np.asarray(X_train)
    y_train = y[~test_idx]
    # Get test data
    X_test = X_[test_idx]
    X_test = np.asarray(X_test)
    y_test = y[test_idx]

    # Normalize train data. Note: Do not normalize overlap! 
    X_train_1 = X_train[:,:-1]
    X_train_2 = X_train[:,-1:]
    mean_1 = np.mean(X_train_1, axis=0,keepdims=True)
    std_1 = np.std(X_train_1, axis=0,keepdims=True)
    X_train_1 = (X_train_1 - mean_1) / (std_1 + 1e-8)
    X_train = np.concatenate([X_train_1,X_train_2],axis=1)
    # Normalize test data. Note: We use train mean+variance. Note: Do not normalize overlap! 
    X_test_1 = X_test[:,:-1]
    X_test_2 = X_test[:,-1:]
    X_test_1 = (X_test_1 - mean_1) / (std_1 + 1e-8)
    X_test = np.concatenate([X_test_1,X_test_2],axis=1)

    # Shuffle train data
    p_train = np.random.permutation(len(y_train))
    X_train = X_train[p_train]
    y_train = y_train[p_train]
    # Shuffle test data
    p_test = np.random.permutation(len(y_test))
    X_test = X_test[p_test]
    y_test = y_test[p_test]
    print('y_train',np.unique(y_train,return_counts=True))
    print('y_test',np.unique(y_test,return_counts=True))

    ########################################
    # TRAINING
    ########################################
    # Create XGBClassifier
    xgb = XGBClassifier(**params)
    # Train XGBClassifier
    train_fit = xgb.fit(X_train, y_train)

    #Deploy XGBClassifier on training set
    dtrain_predprob = xgb.predict_proba(X_train)[:,1]
    print('Train AUC: ', metrics.roc_auc_score(y_train, dtrain_predprob) )
    #Deploy XGBClassifier on test set
    dtest_predprob = xgb.predict_proba(X_test)[:,1]
    auc = metrics.roc_auc_score(y_test, dtest_predprob)
    print('Test AUC: ', auc)

    # Find decision threshold to match FNR just under 10%
    target_fnr_value = 0.1
    fprs, tprs, thresholds = metrics.roc_curve(y_test, dtest_predprob)
    fnrs = 1. - tprs
    idx_my_decision_threshold = np.where( fnrs < target_fnr_value )[0][0]
    decision_threshold = thresholds[idx_my_decision_threshold]
    fpr_value_ = fprs[idx_my_decision_threshold]
    fnr_value_ = fnrs[idx_my_decision_threshold]
    dtest_predprob_binarized = 1.*(dtest_predprob>=decision_threshold)
    tn_, fp_, fn_, tp_ = metrics.confusion_matrix(y_test, dtest_predprob_binarized).ravel()
    print('nb of fp_: ',fp_)
    print('nb of fn_: ',fn_)
    print('fpr_value_: ',fpr_value_)
    print('fnr_value_: ',fnr_value_)
    # print results
    print('The FPR corresponding to a FNR of '+str(fnr_value_)+' is: %.6g' % fpr_value_) 

    # Book keeping
    fnrs_global.append(fnrs)
    fprs_global.append(fprs)
    thresholds_global.append(thresholds)
    fold_global.append(fold)
    auc_global.append(auc)
    fpr_value_global.append(fpr_value_)
    fnr_value_global.append(fnr_value_)

    ########################################
    # Error analysis: plot mispredicted (FP, FN) for error analysis
    ########################################

    # Get FP spots
    idx_FP = np.logical_and(dtest_predprob_binarized==1,y_test==0)
    X_test_FP = X_test[idx_FP]
    list_idx_FP = np.where(idx_FP)
    list_idx_FP = list(list_idx_FP[0])
    print('list_idx_FP',len(list_idx_FP))
    # Get FN spots
    idx_FN = np.logical_and(dtest_predprob_binarized==0,y_test==1)
    X_test_FN = X_test[idx_FN]
    list_idx_FN = np.where(idx_FN)
    list_idx_FN = list(list_idx_FN[0])
    print('list_idx_FN',len(list_idx_FN))

    X_test_ = X_with_helpers_with_overlap[test_idx]
    X_test_ = X_test_.reset_index(drop=True)
    X_test_ = X_test_.reindex(p_test)
    X_test_ = X_test_.reset_index(drop=True)
    # FP
    X_test_FP_ = X_test_[idx_FP]
    X_test_FP_ = X_test_FP_.reset_index(drop=True)
    # FN
    X_test_FN_ = X_test_[idx_FN]
    X_test_FN_ = X_test_FN_.reset_index(drop=True)

    n_FP = X_test_FP.shape[0]
    n_FP_ = min(n_FP,30)
    prev_filename_k = ''
    prev_foldername_k = ''
    # Iterate over FPs
    for k in range(n_FP_):
        # Get meta features
        X_test_FP_k = X_test_FP_.iloc[k]
        foldername_k = X_test_FP_k['foldername']
        filename_k = X_test_FP_k['filename']
        spot_idx_k = X_test_FP_k['spot_idx']
        overlap_k = X_test_FP_k['overlap']
        
        if (foldername_k!=prev_foldername_k) or (filename_k!=prev_filename_k):
            # Load corresponding img
            path_to_patient = os.path.join('..','data',foldername_k)
            path_to_fluo_imgs = os.path.join( path_to_patient,'raw_data','fluorescent' )
            path_to_fluo_img = os.path.join(path_to_fluo_imgs,filename_k)
            if not os.path.isfile(path_to_fluo_img):
                raise ValueError('Image does not exist',foldername_k,filename_k,path_to_fluo_img)
            img_fluo_raw = mpimg.imread(path_to_fluo_img)
            # Rescale image to [0,1]
            img_fluo_raw = img_fluo_raw/255.
            # Crop image 
            x_low,x_high,y_low,y_high = crop_box
            img_fluo_raw = img_fluo_raw[x_low:x_high,y_low:y_high,:]
            # Morphological top-hat transform filtering on each channel of the fluo image
            # We compute the morphological opening of each channel, and then subtracts the result from the original channel
            tophat_filter = np.zeros_like(img_fluo_raw)
            for c in range(img_fluo_raw.shape[-1]):
                tophat_filter[:,:,c] = cv.morphologyEx(img_fluo_raw[:,:,c], cv.MORPH_OPEN, kernel)
            img_fluo = img_fluo_raw - tophat_filter

        # Get pixels of spot
        points = np.asarray( eval(spot_idx_k) )
        ys = points[:,0]; xs = points[:,1]

        # Zoom on the image around spot
        crop_margin = 25
        x_min = np.amin(xs); x_max = np.amax(xs); y_min = np.amin(ys); y_max = np.amax(ys);
        # Get zoomed raw image
        img_fluo_raw_cropped = img_fluo_raw[ max(x_min-crop_margin,0):x_max+crop_margin, max(y_min-crop_margin,0):y_max+crop_margin, :]
        # Get zoomed fluo image
        img_fluo_cropped = img_fluo[ max(x_min-crop_margin,0):x_max+crop_margin, max(y_min-crop_margin,0):y_max+crop_margin, :]

        # Save FP examples to disk
        path_to_dump_FP_viz = os.path.join(".","crossval_results_v3","FP_viz",fold)
        if not os.path.exists(path_to_dump_FP_viz): 
            os.makedirs(path_to_dump_FP_viz)
        # Save raw image 
        plt.close('all')
        fig, ax = plt.subplots(1, 1, figsize=(15,15), sharex=True, sharey=True)
        ax.imshow(img_fluo_raw, interpolation='nearest')
        margin = 1
        c = plt.Rectangle((y_min - margin, x_min - margin), y_max-y_min + 2*margin, x_max-x_min + 2*margin,color=[1.,0,0], lw=1.0, fill=False)
        ax.add_artist(c)
        plt.imshow(img_fluo_raw, alpha=0.6)
        plt.savefig( os.path.join(path_to_dump_FP_viz, fold+'_'+foldername_k+filename_k[:-4]+'_'+str(k)+'_box_raw.png' ))
        # Save zoomed raw image
        plt.close('all')
        fig, ax = plt.subplots(1, 1, figsize=(10,10), sharex=True, sharey=True)
        ax.imshow(img_fluo_raw_cropped, interpolation='nearest')
        margin = 1
        c = plt.Rectangle((crop_margin - margin, crop_margin - margin), y_max-y_min + 2*margin, x_max-x_min + 2*margin,color=[0.5,0,0], lw=1.0, fill=False)
        ax.add_artist(c)
        plt.imshow(img_fluo_raw_cropped, alpha=0.6)
        plt.savefig( os.path.join(path_to_dump_FP_viz, fold+'_'+foldername_k+filename_k[:-4]+'_'+str(k)+'_cropbox_raw.png' ))
        # Save zoomed fluo image
        plt.close('all')
        fig, ax = plt.subplots(1, 1, figsize=(10,10), sharex=True, sharey=True)
        ax.imshow(img_fluo_cropped, interpolation='nearest')
        margin = 1
        c = plt.Rectangle((crop_margin - margin, crop_margin - margin), y_max-y_min + 2*margin, x_max-x_min + 2*margin,color=[0.5,0,0], lw=1.0, fill=False)
        ax.add_artist(c)
        plt.imshow(img_fluo_cropped, alpha=0.6)
        plt.savefig( os.path.join(path_to_dump_FP_viz, fold+'_'+foldername_k+filename_k[:-4]+'_'+str(k)+'_cropbox_fluo.png' ))
        
        # We save the 4 images. We also save the meta data to find the image again
        foldername_k_global.append(foldername_k)
        filename_k_global.append(filename_k)
        spot_idx_k_global.append(spot_idx_k)
        overlap_k_global.append(overlap_k)
            
    # Save foldername_k_global, foldername_k_global, spot_idx_k_global, overlap_k_global to disk
    foldername_k_global = np.reshape(foldername_k_global, [-1,1])
    filename_k_global = np.reshape(filename_k_global, [-1,1])
    spot_idx_k_global = np.reshape(spot_idx_k_global, [-1,1])
    overlap_k_global = np.reshape(overlap_k_global, [-1,1])

    # Book keeping
    if not os.path.exists(os.path.join(path_to_dump_FP_viz, str(fold))): 
        os.makedirs( os.path.join(path_to_dump_FP_viz, str(fold) ) )
    # foldername_k_global
    with open( os.path.join(path_to_dump_FP_viz, str(fold),"FP_foldername_k_global.csv"), "wb") as f:
        writer = csv.writer(f)
        writer.writerows(foldername_k_global)
    # filename_k_global
    with open( os.path.join(path_to_dump_FP_viz, str(fold),"FP_filename_k_global.csv"), "wb") as f:
        writer = csv.writer(f)
        writer.writerows(filename_k_global)
    # spot_idx_k_global
    with open( os.path.join(path_to_dump_FP_viz, str(fold),"FP_spot_idx_k_global.csv"), "wb") as f:
        writer = csv.writer(f)
        writer.writerows(spot_idx_k_global)
    # overlap_k_global
    with open( os.path.join(path_to_dump_FP_viz, str(fold),"FP_overlap_k_global.csv"), "wb") as f:
        writer = csv.writer(f)
        writer.writerows(overlap_k_global)


In [ ]:
# Save results to csv
print( len(fold_global))
print(len(auc_global))
print(len(fpr_value_global))
print(len(fnr_value_global))

fold_global = np.reshape(fold_global, [-1,1])
auc_global = np.reshape(auc_global, [-1,1])
fpr_value_global = np.reshape(fpr_value_global, [-1,1])
fnr_value_global = np.reshape(fnr_value_global, [-1,1])
print(fold_global.shape)
print(auc_global.shape)
print(fpr_value_global.shape)
print(fnr_value_global.shape)

with open( os.path.join(".","crossval_results_v3","fnrs_global.csv") , "wb") as f:
    writer = csv.writer(f)
    writer.writerows(fnrs_global)

with open( os.path.join(".","crossval_results_v3","fprs_global.csv") , "wb") as f:
    writer = csv.writer(f)
    writer.writerows(fprs_global)

with open( os.path.join(".","crossval_results_v3","thresholds_global.csv") , "wb") as f:
    writer = csv.writer(f)
    writer.writerows(thresholds_global)

with open( os.path.join(".","crossval_results_v3","fold_global.csv") , "wb") as f:
    writer = csv.writer(f)
    writer.writerows(fold_global)

with open( os.path.join(".","crossval_results_v3","auc_global.csv") , "wb") as f:
    writer = csv.writer(f)
    writer.writerows(auc_global)

with open( os.path.join(".","crossval_results_v3","fpr_value_global.csv") , "wb") as f:
    writer = csv.writer(f)
    writer.writerows(fpr_value_global)

with open( os.path.join(".","crossval_results_v3","fnr_value_global.csv") , "wb") as f:
    writer = csv.writer(f)
    writer.writerows(fnr_value_global)


In [ ]:
raise ValueError('WIP')

# Plot results

### Load results

In [ ]:
# Load results_v2 from csv

with open( os.path.join(".","crossval_results_v3","fnrs_global.csv")  , 'rb' ) as fp:
    reader = csv.reader(fp, delimiter=',', quotechar='"')
    fnrs_global = [row for row in reader]

with open( os.path.join(".","crossval_results_v3","fprs_global.csv")  , 'rb' ) as fp:
    reader = csv.reader(fp, delimiter=',', quotechar='"')
    fprs_global = [row for row in reader]

with open( os.path.join(".","crossval_results_v3","thresholds_global.csv")  , 'rb' ) as fp:
    reader = csv.reader(fp, delimiter=',', quotechar='"')
    thresholds_global = [row for row in reader]
    
with open( os.path.join(".","crossval_results_v3","fold_global.csv")  , 'rb' ) as fp:
    reader = csv.reader(fp, delimiter=',', quotechar='"')
    fold_global = [row for row in reader]

with open( os.path.join(".","crossval_results_v3","auc_global.csv")  , 'rb' ) as fp:
    reader = csv.reader(fp, delimiter=',', quotechar='"')
    auc_global = [row for row in reader]

with open( os.path.join(".","crossval_results_v3","fpr_value_global.csv")  , 'rb' ) as fp:
    reader = csv.reader(fp, delimiter=',', quotechar='"')
    fpr_value_global = [row for row in reader]

with open( os.path.join(".","crossval_results_v3","fnr_value_global.csv")  , 'rb' ) as fp:
    reader = csv.reader(fp, delimiter=',', quotechar='"')
    fnr_value_global = [row for row in reader]

    
fnrs_global = map( lambda x : map(eval,x), fnrs_global)
fprs_global = map( lambda x : map(eval,x), fprs_global)
thresholds_global = map( lambda x : map(eval,x), thresholds_global)

fold_global = map( lambda x : x[0], fold_global)
auc_global = map( lambda x : eval(x[0]), auc_global)
fpr_value_global = map( lambda x : eval(x[0]), fpr_value_global)
fnr_value_global = map( lambda x : eval(x[0]), fnr_value_global)

print(len(fnrs_global))
print(len(fprs_global))
print(len(thresholds_global))
print(len(fold_global))
print(len(auc_global))
print(len(fpr_value_global))
print(len(fnr_value_global))



### Show results

In [ ]:
# AUC
auc_mean = np.mean(auc_global)
auc_std = np.std(auc_global)
print('auc_mean: ',auc_mean)
print('auc_std: ',auc_std)

In [ ]:
# FPR
fpr_mean = np.mean(fpr_value_global)
fpr_std = np.std(fpr_value_global)
print('fpr_mean: ',fpr_mean)
print('fpr_std: ',fpr_std)

### Identify which AUC curve is the closest to the mean one (e.g. compare AUCs to mean AUC)

In [ ]:
auc_global_mean = np.mean(auc_global)
temp = np.abs(auc_global - auc_global_mean)
curve_closest_to_mean = np.argmin(temp)
curve_closest_to_mean
# curve_closest_to_mean = 3

#### FPR vs FNR curves

In [ ]:
n_folds = len(fnrs_global)
plt.figure( figsize=(15,10) )

# Iterate over the folds, except the mean curve
for i in range(n_folds):
    
    if i==curve_closest_to_mean:
        # we plot the 'mean' curve at the end
        continue

    fnrs = fnrs_global[i]
    fprs = fprs_global[i]
    thresholds = thresholds_global[i]
    fold = fold_global[i]
    assert len(fnrs) == len(fprs)
    assert len(fnrs) == len(thresholds)
    
    # Plot FPR vs FNR on test data
    lw = 1
    plt.plot(fnrs, fprs, color='gray',lw=lw, alpha=0.8)

# Plot the mean curve
i=curve_closest_to_mean
fnrs = fnrs_global[i]
fprs = fprs_global[i]
thresholds = thresholds_global[i]
fold = fold_global[i]
assert len(fnrs) == len(fprs)
assert len(fnrs) == len(thresholds)
plt.plot(fnrs, fprs, color='darkorange',lw=3)

plt.axvline(0.1, c='gray', ls='--')
plt.xlabel('FNR'); plt.ylabel('FPR'); plt.legend(loc="lower right");
plt.yscale('log'); plt.xscale('log'); plt.xlim([0.00001,1.]); plt.ylim([0.0000001,1.]);
plt.title('FPR vs FNR')
plt.show()
    
    


In [ ]:
n_folds = len(fnrs_global)
plt.figure( figsize=(15,10) )

# Iterate over the folds
for i in range(n_folds):
    
    if i==curve_closest_to_mean:
        # we plot the 'mean' curve at the end
        continue

    fnrs = fnrs_global[i]
    fprs = fprs_global[i]
    thresholds = thresholds_global[i]
    fold = fold_global[i]
    assert len(fnrs) == len(fprs)
    assert len(fnrs) == len(thresholds)
    
    ### 
    # Plots for error analysis: we plot the learning curves 
    ###
    # Plot FPR+FNR vs decision thresholds
    plt.plot(thresholds, fprs, color='gray',lw=1, alpha=0.8)#, label='FPR')#+fold)
    plt.plot(thresholds, fnrs, color='gray',lw=1, alpha=0.8)#, label='FNR')#+fold)

    
    
# Plot the mean curve
i=curve_closest_to_mean
fnrs = fnrs_global[i]
fprs = fprs_global[i]
thresholds = thresholds_global[i]
fold = fold_global[i]
assert len(fnrs) == len(fprs)
assert len(fnrs) == len(thresholds)
plt.plot(thresholds, fprs, color='darkorange',lw=3, label='FPR')#+fold)
plt.plot(thresholds, fnrs, color='green',lw=3, label='FNR')#+fold)
    
plt.axhline(0.1, c='green', ls='--')
plt.xlabel('thresholds'); plt.ylabel('FPR and FNR');plt.legend(loc="lower right")
plt.yscale('log'); plt.xlim([0.,1.1]); plt.ylim([0.,1.1]);
plt.show()

